### Loading Required Libraries

In [1]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn
import spacy
# import spacy.cli
# spacy.cli.download("en_core_web_md")
import spacy_fastlang
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from bertopic import BERTopic
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sqlite3 import connect 
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

In [4]:
databse_file = 'database.db'
main = connect(databse_file)

sql_statement_laod_data = '''
SELECT 
    reviews.id, 
    reviews.course_name,
    reviews.text

FROM reviews
'''

df = pd.read_sql_query(sql_statement_laod_data, main)
df.sample(5)

,id,course_name,text
24070,24070,foundations-data,Very good contents and the instructor makes it...
38257,38257,python-network-data,Excellent\n
52454,52454,learning-how-to-learn,I loved this course! It helped me a lot in th...
22754,22754,foundations-data,Gâreat course and gives alots of insight eve...
56219,56219,learning-how-to-learn,I found this course to be a simple and effecti...


In [5]:
df.isnull().sum() * 100 / len(df)

id             0.000000
course_name    0.000000
text           0.049845
dtype: float64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum() * 100 / len(df)

id             0.0
course_name    0.0
text           0.0
dtype: float64

In [8]:
import re
def text_decoder(string):
    '''
        The following two lines of code are from Wim Feijen on StackOverFlow

        Wim Feijen
        March 26, 2021
        StackOverFlow: Python encoding/decoding problems
        https://stackoverflow.com/questions/27996448/python-encoding-decoding-problems
    '''
    bytes_string = bytes(string, encoding="raw_unicode_escape")
    text = bytes_string.decode("utf-8", "strict")
    return text

def lang_detect(clean_string):
    clean_string = nlp(clean_string)
    return clean_string._.language

def tokens_2_keep(text_column):
    tokens = text_column.split()
    return [token for token in tokens if re.match('^[a-z0-9]+$', token)]

nlp = spacy.load('en_core_web_md', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
nlp.add_pipe("language_detector")

In [9]:
# filter out any non-english reviews
df['language'] = df['text'].apply(lang_detect)
df = df[df['language'] == 'en']
df['cleaned_text'] = df['text'].apply(text_decoder)
df.sample(2)

In [19]:
def token_counter(text):
    return len(text.split())

df['token_count'] = df['cleaned_text'].apply(token_counter)
# grab docs > 4 tokens
df = df.loc[df['token_count'] > 4,:]
df = df[['course_name', 'cleaned_text', 'token_count']]
df.sample(2)

It looks like we are fine from overfitting to the single word documents/reviews, but to stay safe we will create two different models.  One with single word docs and one without single word docs.

# Model

In [22]:
count_text_vectorizer = CountVectorizer(stop_words=stopwords, min_df=0.1, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(df["cleaned_text"])

lda_para_model = LatentDirichletAllocation(n_components=6 , random_state=1)
lda_para_model.fit(count_text_vectors)

lda_display = pyLDAvis.sklearn.prepare(lda_para_model, count_text_vectors, count_text_vectorizer, sort_topics=False);

/opt/anaconda3/envs/dsOne/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/dsOne/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/dsOne/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/dsOne/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/dsOne/lib/python3.9/site-packages/past/builtins/misc.py:45: 

In [23]:
pyLDAvis.display(lda_display)
